# Representing syntax
Julia's metaprgoramming features are based on Lisp (apparently) -- a language and sytax born from mathematical representations of a language. 

Code can be represented as a tree structure where the nodes are literals (values), other expressions or symbols (variables?, more on this later).

       + 
      /|\
     / | \
    1  2  *
         / \
        /   \
       3     4

(*, 3, 4)

(+, 1, 2, (*, 3, 4))    

This is exactly what Julia does. 

--- 

Source code in Julia is a text file -- a .jl file is just a text file with a .jl extension.
As the first step in running Julia code, it is parsed from a string into a datastructure called an Expression (`Expr()`)


In [2]:
source_code = "1+2"

"1+2"

In [4]:
expr = Meta.parse(source_code)

:(1 + 2)

In [5]:
Meta.show_sexpr(expr)

(:

call, :+, 1, 2)

This is (apparently) the same as what LISP does. In contrast, C, represents code as a string which can be manipulated and then parsed, rather than as a data structure. 

--- 

# `Expr`essions

```julia
mutable struct Expr
    head::Symbol
    args::Array{Any,1}
end
```
(The source for `Expr` is implemented directly in C.)

(why mutable??)

In [6]:
typeof(expr)

Expr

In [7]:
fieldnames(Expr)

(:head, :args)

In [8]:
dump(expr)

Expr
  head: Symbol call
  args: Array{Any}(

(3,))
    1: Symbol +
    2: Int64 1
    3: Int64 2


## `expr.head`
The `expr.head` is a `Symbol` -- An *interned* string which specifies the *type* of expression.

---
Aside 
```
mutable struct Symbol
#    opaque
end
```
The source code for `Symbol` is actually directly in C. The comment `# opaque` refers to the fact that the innards of `Symbol` are hidden from the user. 
*The type of object used to represent identifiers in parsed julia code (ASTs). Also often used as a name or label to identify an entity (e.g. as a dictionary key)*

---

Examples. 

    - `:call` - call a callable object (function)

    - `:curly` - wrap args in curly braces

    - `:quote` - quote julia code

    - `:block` - begin/end

    - `:tuple` - make a tuple

    - `:vect` - construct a vector from args

    - `:row` - construct a row from args

    - `:vcat`, `:hcat` - concatenation

    - `:macrocall` - same as call but for macros

    - `:ref` - reference an array 

    - `:...` - splat operator 

    - `:if`, `:for`, `:while`
    
    - `:using`, `:import`

## expr.args

Arguments -- can be more expressions, literals (i.e. the value 1.0), symbols. 

In [32]:
y = 2
expr = Expr(:call, :+, :x, y)

:(x + 2)

In [34]:
expr2 = Expr(:call, :+, :x, :y)

:(x + y)

In [35]:
expr3 = Expr(:call, :+, 1.0, 2.0, 3.0, x, :x)

:(1.0 + 2.0 + 3.0 + 1.0 + x)

In [28]:
array_sexpr = Expr(:call, Expr(:curly, :Array, :Float64,1), :undef, 2)

:(Array{Float64, 1}(undef, 2))

In [29]:
array_source = Meta.parse("Array{Float64,1}(undef,2)")

:(Array{Float64, 1}(undef, 2))

In [30]:
array_source==array_sexpr

true

# Quoting
S-expressions/the `Expr` constructor are unwieldy

In [25]:
expr = Expr(:call, :+, 1.0, :x)

:(1.0 + x)

Notice symbol `:x` vs just `x` below, both are NOT a value -- `x` is not even defined yet.

In [26]:
quoted_expr = :(1.0+x)

:(1.0 + x)

In [27]:
block_expr = quote 
    1.0+x
end

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:2 =#
    1.0 + x
end

In [28]:
quoted_expr==expr

true

In [29]:
quoted_expr==block_expr

false

In [30]:
dump(expr)

Expr
  head: Symbol call
  args: Array{Any}(

(3

,))
    1: Symbol +
    2: Float64 1.0
    3: Symbol x


In [31]:
dump(quoted_expr)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Float64 1.0
    3: Symbol x


In [32]:
dump(block_expr)


Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 

2
      file: Symbol /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Float64 1.0
        3: Symbol x


In [33]:
expr.args

3-element Vector{Any}:
  :+
 1.0
  :x

In [34]:
block_expr.args

2-element Vector{Any}:
 :(#= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:2 =#)
 :(1.0 + x)

`Expr` can be built from pretty much anything but will throw an error at run time if it represents undefined syntax. But there is no forewarning.

In [42]:
expr2 = Expr(:call, 1, [],)
# eval(expr2) #nonesense 

:((1)(Any[]))

In [44]:
Expr(:vcat, :udef_var, :for)

:([udef_var; for])

---

# Interpolation

Often we want to include a value in an expression via a variable. i.e. 

In [47]:
x = 0.0000000000004872889457363563107187 # something annoying to type 

4.872889457363563e-13

In [48]:
Expr(:call, :*, x, 1e12)
# as opposed to 
# Expr(:call, :*, :x, 1_000_000_000_000)

:(4.872889457363563e-13 * 1.0e12)

Straightforward with `Expr`, but less so for quoting;

In [113]:
:(x*1e12) # x here is a variable name

:(x * 1000000000000)

In [114]:
:($x*1e12) # x here is the value 0.0000000000004872889457363563107187

:(4.872889457363563e-13 * 1000000000000)

Purposely reminiscent of string interpolation.

In [49]:
expr = quote 
    y = $x
    for n in 1:12
        global y = y*10 
    end
    y
end

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:2 =#
    y = 4.872889457363563e-13
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:3 =#
    for n = 1:12
        #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:4 =#
        global y = y * 10
    end
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:6 =#
    y
end

You can nest quote expressions and interpolate into nested expressions. I've never needed to. Interpolation is weird with nested quotes -- see the docs. 

`QuoteNode` is a quote without interpolation -- but no nice syntax for this yet.

---

# Evaluating expressions and scoping 

Every module has an `eval` function to evaluate expressions

``` 
    eval(expr)
```
*Evaluate an expression in the global scope of the containing module. Every Module (except those defined with baremodule) has its own 1-argument definition of eval, which evaluates expressions in that module.*

In [50]:
expr = :(1.0+1.0)
eval(expr)

2.0

With interpolation:

In [51]:
z = 1.0
expr = :($z+1.0)

:(1.0 + 1.0)

In [54]:
eval(expr)

2.0

In [56]:
z=10
expr # no change to expr

:(1.0 + 1.0)

In [57]:
eval(expr) # z has already been interpolated, so expr does not depend on z

2.0

There is also a macro for interpolation

In [58]:
z = 1.0
@eval $z+1 

2.0

There is a difference between interpolation and the value of a variable at run-time

In [59]:
z = 1.0
expr = :(z+1.0) # expr is a function of z

:(z + 1.0)

In [60]:
expr_interp = :($z+1.0) # not a function of z

:(1.0 + 1.0)

In [61]:
z = 10
eval(expr) # expr is a function of z

11.0

In [62]:
eval(expr_interp)

2.0

### ~~Scoping can be weird!~~


In [63]:
expr = quote
    q = $z
    for n in 1:10 
        q *= 10
    end
    q
end

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:2 =#
    q = 10
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:3 =#
    for n = 1:10
        #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:4 =#
        q *= 10
    end
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:6 =#
    q
end

In [64]:
eval(expr)

# ┌ Warning: Assignment to `q` in soft scope is ambiguous because a global variable by the same name exists: `q` will be treated as a new local. Disambiguate by using `local q` to suppress this warning or `global q` to assign to the existing global variable.
# └ @ REPL[83]:4
# ERROR: UndefVarError: q not defined
# Stacktrace:
#  [1] top-level scope
#    @ ./REPL[83]:4

UndefVarError: UndefVarError: q not defined

I thought this would work becuase the following does 

In [65]:
q = z
for n in 1:10 
    q *= 10
end
q

100000000000

~~The issue appears to be that the for loop creates a new scope and hence new variable `q`.~~

This is a REPL weirness. The loop above throws the same error as the `expr` when run via the terminal. 

In [66]:
expr = quote
    q = $z
    for n in 1:10 
        global q *= 10
    end
    q
end

quote
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:2 =#
    q = 10
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:3 =#
    for n = 1:10
        #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:4 =#
        global q *= 10
    end
    #= /Users/anguslewis/Documents/Julia_Meta_Talk/meta.ipynb:6 =#
    q
end

In [67]:
eval(expr)

100000000000

### Scoping with modules.

Each module has its own 1-argument `eval` function to evaluate expressions within the scope of that module.

In [69]:
# Main module 
x = 10 
y = 3
expr = :(x+y)

:(x + y)

In [70]:
module MyModule
    x = 33
    y = 1000
end

Main.MyModule

In [71]:
Main.eval(expr)

13

In [72]:
MyModule.eval(expr)

1033

### Side effects

In [73]:
x

10

In [74]:
ex = :(x=1)

:(x = 1)

In [75]:
eval(x)

10

In [76]:
x # in Main module

10

In [77]:
MyModule.x # x in MyModule has not changed

33

In [78]:
MyModule.eval(ex) # now it has, even though ex was defined in Main 

1

In [79]:
MyModule.x

1

# Functions on expressions 

In [86]:
# stolen from the docs 
function double_numeric_args(binary_operation, arg1, arg2)
    if isa(arg1,Number)
        arg1 = 2*arg1
    end
    if isa(arg2,Number)
        arg2 = 2*arg2
    end
    return Expr(:call, binary_operation, arg1, arg2)
end

double_numeric_args (generic function with 1 method)

In [89]:
double_numeric_args(:+, 1, 3)

:(2 + 6)

In [90]:
double_numeric_args(:+, 1, :(1+2))

:(2 + (1 + 2))

## Other useful applications

Defining a suite of unary operations on a new type

In [91]:
struct my_angle
    a::Float64
end

for f in (:sin, :cos, :tan, :tanh, :cosh, :sinh, :exp)
    @eval Base.$f(x::my_angle) = $f(x.a)
end


In [92]:
x = my_angle(10.0)

my_angle(10.0)

In [93]:
sin(x)

-0.5440211108893698

Defining a suite binary operations for various other concrete types

In [1]:
struct my_matrix{T} <: AbstractArray{T,2} 
    a::Array{T,2}
end

In [2]:
concrete_types = (:Float64, :Int, :Complex)
funs = (:+,:-,:*,:/,:\)
for type in concrete_types
    for f in funs
        @eval Base.$f(m::my_matrix,v::$type) = $f(m.a,v)
        @eval Base.$f(v::$type,m::my_matrix) = $f(m.a,v)
        @eval Base.$f(m::my_matrix,v::Array{$type,2}) = $f(m.a,v)
        @eval Base.$f(v::Array{$type,2},m::my_matrix) = $f(m.a,v)
    end
end

Although we should probably just define `Base.getindex`, `Base.size` then we would get all the `funs` for free by inheriting from `AbstractArray`

In [9]:
# Base.size(x::my_matrix) = size(x.a)
# Base.getindex(x::my_matrix,i) = x.a[i]
# Base.getindex(x::my_matrix,i,j) = x.a[i,j]

In [3]:
M = my_matrix{Float64}([1.0 2.0; 3.0 4.0]);
M\[1.0 0.0; 0.0 1.0]

2×2 Matrix{Float64}:
 -2.0   1.0
  1.5  -0.5

# Macros

Macros execute at *parse time* (not comple time or run time). They allow you to generate and include fragments of code before the program is run.

In [1]:
macro slow_calc(x)
    z = sum(collect(1:1000_000_000)) # big calc
    return :($(esc(x))+$z)
end

@slow_calc (macro with 1 method)

In [21]:
@macroexpand @slow_calc(1.0)

:(1.0 + 500000000500000000)

In [22]:
function f_slow(x)
    z = sum(collect(1:1000_000_000))
    return x+z
end

f_slow (generic function with 1 method)

In [23]:
function f_fast(x)
    return @slow_calc(x)
end

f_fast (generic function with 1 method)

In [24]:
f_slow(1.0)

5.000000005e17

In [5]:
f_fast(1.0)

5.000000005e17

In [12]:
function f_slow2(x)
    return x+eval(:(sum(collect(1:1000_000_000))))
end

f_slow2 (generic function with 1 method)

In [16]:
function f_slow3(x)
    return eval(:($x+sum(collect(1:1000_000_000))))
end

f_slow3 (generic function with 1 method)

In [17]:
f_slow2(1.0)

5.000000005e17

In [18]:
f_slow3(1.0)

5.000000005e17

# stolen!

Macros
Macros are special functions that transform expressions and insert the result back into your code.

This can be useful for a variety of reasons.

A convenient way to "annotate" expressions, like @inline f(x) = ...
Provide tools that take expressions and use them for a task - @time, @code_typed, @benchmark, etc
Create a useful shortcut to save on typing - Base.@nloops makes arbitry number of nested loops
Create an entire DSL - like Query.jl, Flux.jl, etc.

https://github.com/FugroRoames/RoamesNotebooks/blob/master/A%20practical%20introduction%20to%20metaprogramming%20in%20Julia.ipynb

https://gist.github.com/fcard/b735e642e6613feffad2d00f3c4298bd

https://stackoverflow.com/questions/58137512/why-use-macros-in-julia

https://github.com/cstjean/Unrolled.jl

In [ ]:
y

0.4872889457363563